<a href="https://colab.research.google.com/github/ebayuk/singtomeabelrepo/blob/main/singtomeabel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##trying to figure out how to extract only the text from the weeknd json downloaded from hugging face so can then follow the video tutorial to create genai model

In [19]:
with open('/theweeknd.txt', 'r', encoding = 'utf-8', newline='\n') as f:
  text = f.read()

In [49]:
for i in text2[0:2]:
    print(i)

 
Y


In [73]:
import re

regex = re.compile('[^a-zA-Z]')
text2 = regex.sub(' ', text)

In [70]:
import re

regex = re.compile('[!"#$%&()*,-./0123456789:;=?[\n^{|}\´ÀÁÇÜàáçèéêìíñòóôöùúüąćęğİıłńœśŞşźżƏəАСУХЧШабвгдежзийлмнорстухчшыьэүө–—‘’“”… イオガトブメラ]')
text3 = regex.sub(' ', text)

In [84]:
#trying to split out all the special characters

newtext = []
i = 0

for count, value in enumerate(text2):
  if value == 'n' and count+1 value
    newtext.append(value+1)
  else:
    newtext.append(value)

In [80]:
#same as above
chars2 = chars[1:]

In [86]:
chars = sorted(list(set(text2)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
53


In [87]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hi there'))
print(decode(encode('hi there')))

[34, 35, 0, 46, 34, 31, 44, 31]
hi there


In [89]:
#encode datset into torch.tensor
import torch
data = torch.tensor(encode(text2), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([896504]) torch.int64
tensor([ 0, 25, 31, 27, 34,  0, 40, 25, 31, 27, 34,  0, 40, 25, 31, 27, 34,  0,
        40, 25, 41, 47, 44,  0, 39, 27, 40,  0, 41, 40,  0, 46, 34, 31,  0, 44,
        41, 27, 30,  0,  0, 34, 31,  0, 30, 41, 35, 40,  0, 42, 44, 41, 39, 41,
         0, 40, 25, 41, 47,  0, 45, 27, 35, 30,  0,  0,  0,  0, 11, 31, 31, 42,
         0, 41, 47, 44,  0, 28, 47, 45, 35, 40, 31, 45, 45,  0, 41, 40,  0, 46,
        34, 31,  0, 38, 41, 49,  0, 38, 41, 49,  0,  0,  0, 40,  9, 39,  0, 36,
        47, 45, 46,  0, 46, 44, 51, 40, 27,  0, 33, 31, 46,  0, 51, 41, 47,  0,
        41, 47, 46,  0, 46, 34, 31,  0, 32, 44, 35, 31, 40, 30,  0, 52, 41, 40,
        31,  0, 40,  3, 27, 47, 45, 31,  0, 51, 41, 47,  0, 38, 41, 41, 37,  0,
        31, 48, 31, 40,  0, 28, 31, 46, 46, 31, 44,  0, 46, 34, 27, 40,  0, 46,
        34, 31,  0, 42, 34, 41, 46, 41, 45,  0, 40,  9,  0, 29, 27, 40, 46,  0,
        32, 35, 40, 30,  0, 51, 41, 47, 44,  0, 34, 41, 47, 45, 31,  0,  0, 45,
       

In [90]:
#split data into train test
n = int((0.9)*len(data))
traindata = data[:n]
val_data = data[n:]

In [91]:
block_size = 8
traindata[:block_size+1]

tensor([ 0, 25, 31, 27, 34,  0, 40, 25, 31])

In [93]:
#how we will train the data
x = traindata[:block_size]
y = traindata[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when input is {context} the target: {target}')

when input is tensor([0]) the target: 25
when input is tensor([ 0, 25]) the target: 31
when input is tensor([ 0, 25, 31]) the target: 27
when input is tensor([ 0, 25, 31, 27]) the target: 34
when input is tensor([ 0, 25, 31, 27, 34]) the target: 0
when input is tensor([ 0, 25, 31, 27, 34,  0]) the target: 40
when input is tensor([ 0, 25, 31, 27, 34,  0, 40]) the target: 25
when input is tensor([ 0, 25, 31, 27, 34,  0, 40, 25]) the target: 31


In [97]:
#going to feed batches of these chunks of texts into the transformer
torch.manual_seed(1333)
batch_size = 4 #how many independent sequences will we process in parallel
block_size = 8 #what is the maximum context length for predictions

def get_batch(split):
  #generate a small batch of data of inputs x and targets y
  data = traindata if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f'when input is {context.tolist()} the target: {target}')


inputs:
torch.Size([4, 8])
tensor([[47,  0, 39, 27, 37, 31,  0, 35],
        [31, 46, 31, 38, 51,  0, 41, 32],
        [51, 41, 47,  0, 28, 35, 46, 31],
        [35, 41, 40, 45,  0, 49, 31, 44]])
targets:
torch.Size([4, 8])
tensor([[ 0, 39, 27, 37, 31,  0, 35, 46],
        [46, 31, 38, 51,  0, 41, 32, 32],
        [41, 47,  0, 28, 35, 46, 31,  0],
        [41, 40, 45,  0, 49, 31, 44, 31]])
----
when input is [47] the target: 0
when input is [47, 0] the target: 39
when input is [47, 0, 39] the target: 27
when input is [47, 0, 39, 27] the target: 37
when input is [47, 0, 39, 27, 37] the target: 31
when input is [47, 0, 39, 27, 37, 31] the target: 0
when input is [47, 0, 39, 27, 37, 31, 0] the target: 35
when input is [47, 0, 39, 27, 37, 31, 0, 35] the target: 46
when input is [31] the target: 46
when input is [31, 46] the target: 31
when input is [31, 46, 31] the target: 38
when input is [31, 46, 31, 38] the target: 51
when input is [31, 46, 31, 38, 51] the target: 0
when input is [31, 4

In [104]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1333)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets):
    #logits = score for next token in the sequence
    logits = self.token_embedding_table(idx) #B, T, C

    B,T,C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)
    loss = F.cross_entropy(logits, targets)

    return logits, loss



m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 53])
tensor(4.4319, grad_fn=<NllLossBackward0>)
